# 1. Configure GeoLib to use Girder & NERSC resources

In [1]:
# Get NEWT session id
import os
newt_sessionid = open(os.path.expanduser('~/temp/newt_sessionid.txt')).read().strip()
nersc_repository = None

In [2]:
# Load GeoLib and connect to girder using NEWT session id
import geolib
girder_url = 'http://turtleland4:8085'
girder = geolib.connect(girder_url=girder_url, newt_sessionid=newt_sessionid)


{'output': '/global/cscratch1/sd/johnt', 'error': '', 'status': 'OK'}


# 2. Select dataset from ESS-DIVE

In [3]:
girder.ls('/user/admin/Public')

[['2.1 MB', 'PH_community_distribution_map.tif'],
 ['8.5 MB', 'PH_plant_height_map.tif'],
 ['8.2 KB',
  'Remote Sensing and Geophysical Characterization of a Floodplain-Hillslope System in the East River Watershed, Colorado.xml']]

In [4]:
# Select item from ESS-DIVE

girder_path = '/user/admin/Public/PH_community_distribution_map.tif'
girder_url = girder.lookup_url(girder_path)
essdive_object = geolib.create(girder_url)

In [5]:
# Fix metadata
essdive_object._epsg = 4326
essdive_object._setdatatype(geolib.types.RASTER)
if girder_path.split('/')[-1] in ['PH_community_distribution_map.tif', 'PH_plant_height_map.tif']:
    metadata = essdive_object.get_metadata()
    essdive_object._metadata['bounds']['coordinates'] = [[
        [-106.95242631255823, 38.92484206381101],
        [-106.95224175189571, 38.91810990322611],
        [-106.94362420673949, 38.91825374356009],
        [-106.94380795397146, 38.92498593850482]
    ]]

essdive_object.opacity = 0.5
geolib.show(essdive_object)

scene(center={'x': 0.0, 'y': 0.0}, layout=Layout(align_self='stretch', height='400px'))

# 3. Define Crop Geometry

In [6]:
# Create polygon to display bounds
bounds = essdive_object.get_metadata().get('bounds')
coordinates = bounds.get('coordinates')
import geojson
bounds_geometry = geojson.Polygon(coordinates)
bounds_feature = geojson.Feature(geometry=bounds_geometry, properties={'fillColor': 'black', 'fillOpacity': 0.2})
bounds_object = geolib.create(bounds_feature)

# Generate small crop geometry
coords = coordinates[0]
xvals, yvals = zip(*coords)
xmin = min(xvals); ymin = min(yvals)
xmax = max(xvals); ymax = max(yvals)
x = (xmin + xmax) / 2.0
y = (ymin + ymax) / 2.0

# Use small percentage of height & width
dx = 0.12 * (xmax - xmin)
dy = 0.16 * (ymax - ymin)
poly = [[
    [x,y], [x+dx,y+dy], [x-dx,y+dy], [x-dx,y-dy], [x+dx,y-dy]
]]
crop_geometry = geojson.Polygon(poly)
crop_feature = geojson.Feature(geometry=crop_geometry, properties={'fillColor': 'magenta', 'fillOpacity': 0.5})
crop_object = geolib.create(crop_feature)

scene1 = geolib.show([crop_object, bounds_object])
display(scene1)

scene(center={'x': 0.0, 'y': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [7]:
import ipywidgets as widgets
slider1 = widgets.FloatSlider(min=1, max=16, value=scene1.zoom, layout=dict(width='95%'))
newlink = widgets.jslink((slider1, 'value'), (scene1, 'zoom'))
display(slider1)

FloatSlider(value=15.5, layout=Layout(width='95%'), max=16.0, min=1.0)

# 4. Submit preprocesssing job on NERSC (Cori)

In [ ]:
# To run on local machine or Girder:
# output_object = geolib.crop(essdive_object, crop_object)
nersc_repository = None
# To run on NERSC:
import getpass
while not nersc_repository:
    nersc_repository = getpass.getpass('Enter NERSC repository (account): ')
cori_job = geolib.submit_crop(essdive_object, crop_object, nersc_repository)
cori_job

# 5. Use job id to create GeoLib object

In [9]:
job_id = cori_job

In [ ]:
job_id = '5c93c50ded1b6319b488e3dc'  # (PH_community_distribution_map.tif)

girder_url2 = girder.lookup_url(job_id=job_id)
output_object = geolib.create(girder_url2, bounds=[x-dx,y-dy,x+dx,y+dy])
# print('output_object', output_object)

# Display cropped dataset (must again set metadata)
output_object._epsg = 4326
output_object._setdatatype(geolib.types.RASTER)
output_object.opacity = 0.8
scene2 = geolib.show(output_object)
display(scene2)

In [10]:
import ipywidgets as widgets
slider2 = widgets.FloatSlider(min=1, max=20, value=scene2.zoom, layout=dict(width='95%'))
newlink = widgets.jslink((slider2, 'value'), (scene2, 'zoom'))
display(slider2)

FloatSlider(value=15.5, layout=Layout(width='95%'), max=20.0, min=1.0)